In [ ]:
__author__ = "Jose David Marroquin Toledo"
__credits__ = ["Jose David Marroquin Toledo",]
__email__ = "jose@marroquin.cl"
__status__ = "Development"

## Simulate Image Sets for Fourier Ptychography
With these functions, it is possible to generate **Lo-Res image sets to be used with a Fourier Ptychography algorithm**. We suggest to read the [fpblend-examples notebook](../examples/fpblend-examples.ipynb) and follow their steps.

[blendjupyter.ipynb](blendjupyter.ipynb) is a shared Jupyter module &#171;written to set up and manipulate cameras (`'bpy.types.Camera'`), scenes (`'bpy.context.scene'`)&#187;, among other things.

This notebook **does not use** a Python kernel, [uses a Blender kernel](https://github.com/panzi/blender_ipython).

In [ ]:
import bpy
import math
import blendjupyter as blendj

In [ ]:
def gen_xyz_ring_led(**kwargs):
    """Returns a list with the coordinates of each LED of a LED ring.
    """
    d = kwargs.pop('d', 5)  # Distance in mm between LEDs.
    # Number of LEDs per ring from the center of it.
    ledsperring = kwargs.pop('ledsperring', [12, 18, 36])
    h = kwargs.pop('h', 90)  # Distance in mm between the sample
                             # and the grid.
    l_coord = list()
    for nleds in ledsperring:
        r = (d * nleds) / (2.0 * math.pi)
        theta_step = int(360 / nleds)
        range_theta = range(0, 360, theta_step)
        range_theta = range_theta[:nleds]
        for theta in range_theta:
            x = r * math.cos(math.radians(theta))
            y = r * math.sin(math.radians(theta))
            z = h
            l_coord.append((x, y, z))
    return l_coord

In [ ]:
def gen_xyz_led_grid(**kwargs):
    """Returns a list with the coordinates of each LED of a n-by-n LED
    grid."""
    n = kwargs.pop('n', 15)  # Number of LEDs per row or per column in
                             # the grid LED.
    d = kwargs.pop('d', 4)  # Distance in mm between LEDs.
    h = kwargs.pop('h', 90)  # Distance in mm between the sample
                             # and the grid.
    l_coord = list()
    x_max = math.floor((n / 2)) * d
    x_min = -x_max
    y_max = x_max
    y_min = x_min
    l_range = list(range(x_min, x_max + 1, d))
    for i in l_range:
        for j in l_range:
            l_coord.append((j, -i, h))  # (x, y, z)
    return l_coord

In [ ]:
def create_led_illuminator(sc, l_coord, **kwargs):
    """Returns a list with the 'LAMPS' of the LED illuminator."""
    radiusled = kwargs.pop('radiusled', 1.25)  # Radius in mm of the
                                               # LED.
    coordcam = kwargs.pop('coorcam', (0, 0, l_coord[0][2] - 1))
    l_lamps = list()
    len_lcoord = len(l_coord)
    for i in range(len_lcoord):
        lamp_name = "lamp" + blendj.num_str_zeros(i,
                                                  len_lcoord,
                                                  matlab=True)
        # The lamp as a point.
        lamp_data = bpy.data.lamps.new(name=lamp_name,
                                       type='POINT')
        lamp_data.energy = 0.0 # The lamp if off.
        lamp_object = bpy.data.objects.new(name=lamp_name,
                                           object_data=lamp_data)
        sc.objects.link(lamp_object)
        lamp_object.location = l_coord[i]
        lamp_object.select = True
        sc.objects.active = lamp_object
        # Base model for the LED.
        bpy.ops.mesh.primitive_cylinder_add(location=l_coord[i],
                                            radius=radiusled,
                                            depth=0)
        l_lamps.append(lamp_object)
    return l_lamps

In [ ]:
def illuminate_step_by_step(cam, sc, l_lamps, **kwargs):
    """Turns on a lamp at time, take a photo of the mesh and save it.
    """
    len_llamps = len(l_lamps)
    path = blendj.find_out_dir(dirname='blend-lores-set-0001',
                               parentdir='microscope')
    for i in range(len_llamps):
        l_lamps[i].data.energy = 2.0
        blendj.shoot_cam(cam,
                         sc,
                         i,
                         len_llamps,
                         path,
                         prefix='lores-img_')
        l_lamps[i].data.energy = 0.0